In [ ]:
#define function   
import numpy as np 
import matplotlib.pyplot as plt 
from keras import Input, layers, activations, backend ,optimizers
from keras.models import  Model
import tensorflow as tf



def ricker(f0,t):
    f=f0
    r=-(np.pi*f*(t-0.1))**2
    return (1+2*r)*(np.e)**r


def velocity_profile(size,npml,dx):
    layer=[220,450,700,]  #  position of boundary [meter] 
    velocity=[2000,3000,4000,5000] # m/s  
    nx=int((size+2*npml)/dx)
    layer=np.array(layer)+npml
    v=np.zeros(nx)
    k0=0
    for j,i in enumerate(layer):
        k1=int(i/dx)
        v[k0:k1]=velocity[j]
        k0=k1
        
    v[k0:]=velocity[j+1]  
    return v


def data_gen(f0,totaltime,Dimension,PMLsize,dx,rp,sp,timeskip,v):
    global a_x,b_x,a_x_half,b_x_half,energy

    TT=totaltime #time (second)

    Dimension=Dimension+2*PMLsize
    nodepml= int(PMLsize/dx)


    nx=int(np.round(Dimension/dx))
    #v = velocity_profile(nx,PMLsize) 
    vmax=max(v)
    dt=dx/vmax
    NT=int(np.round(TT/dt))
    
    energy=np.zeros(NT)

    # PML 
    Xorigin_left= PMLsize #size of PML (meter)
    Xorigin_right=(nx-1)*dx-Xorigin_left

    ALPHA_MAX_PML=np.pi*f0;
    #NN=2
    #Rc=0.001
    #d0_x=-(NN+1)*vmax*np.log(Rc)/(2*Xorigin_left)
    d0_x=9*vmax/(2*Xorigin_left)

    d_x=np.zeros(nx)
    d_x_half=np.zeros(nx)
    #K_x=np.ones(nx)
    #K_x_half=np.ones(nx)
    alpha_x=np.zeros(nx)
    alpha_x_half=np.zeros(nx)
    a_x=np.zeros(nx)
    a_x_half=np.zeros(nx)
    b_x=np.zeros(nx)
    b_x_half=np.zeros(nx)


    #saving data
    Y=[]  #wave at reciever
    So=np.zeros([NT]) #source


    # reciever positions 
    rp=rp+PMLsize  
    rp=int(rp/dx)

    # sorce position

    sp=sp+PMLsize 
    Sp=np.zeros(nx)
    Sp[int(sp/dx)]=1

    # prepare coff
    for i in range(nx):
        xval=i*dx

        #Xmin
        absci=Xorigin_left-xval
        if absci >=0 :
            abscissa_normalized=absci/Xorigin_left
            d_x[i]=d0_x*(abscissa_normalized**2) 
            alpha_x[i]=ALPHA_MAX_PML*(1-abscissa_normalized)

        absci=Xorigin_left-(xval+dx/2)
        if absci >=0 :
            abscissa_normalized=absci/Xorigin_left
            d_x_half[i]=d0_x*(abscissa_normalized**2)
            alpha_x_half[i]=ALPHA_MAX_PML*(1-abscissa_normalized)

        #Xmax
        absci=xval-Xorigin_right
        if absci >=0 :
            abscissa_normalized=absci/Xorigin_left
            d_x[i]=d0_x*(abscissa_normalized**2)
            alpha_x[i]=ALPHA_MAX_PML*(1-abscissa_normalized)

        absci=xval+dx/2-Xorigin_right
        if absci >=0 :
            abscissa_normalized=absci/Xorigin_left
            d_x_half[i]=d0_x*(abscissa_normalized**2)
            alpha_x_half[i]=ALPHA_MAX_PML*(1-abscissa_normalized)


        b_x[i] = np.exp(- (d_x[i]  + alpha_x[i]) * dt)
        b_x_half[i] = np.exp(- (d_x_half[i]  + alpha_x_half[i]) * dt)
        
       
        if abs(d_x[i]) >1e-6 :
            #a_x[i] = d_x[i] * (b_x[i] - 1) / (K_x[i] * (d_x[i] + K_x[i] * alpha_x[i]))
            a_x[i] = d_x[i] * (b_x[i] - 1) / (d_x[i] +  alpha_x[i])
        if abs(d_x_half[i]) > 1e-6: 
            #a_x_half[i] = d_x_half[i] * (b_x_half[i] - 1) / (K_x_half[i] * (d_x_half[i] + K_x_half[i] * alpha_x_half[i]))
            a_x_half[i] = d_x_half[i] * (b_x_half[i] - 1) / (d_x_half[i] + alpha_x_half[i])
    mpx=np.zeros(nx)
    mpxx=np.zeros(nx)
    vpx=np.zeros(nx)
    vpxx=np.zeros(nx)
    pxx=np.zeros(nx)
    dpxx=np.zeros(nx)
    u0=np.zeros(nx)
    u1=np.zeros(nx)
    u2=np.zeros(nx)

    kappa=(v**2)*dt*dt/dx/dx
    vsource=kappa[int(sp/dx)]*dx*dx
    
    iii=0
    for it in range(NT):
        
        So[it]=ricker(f0,it*dt)*vsource
        
        if it%timeskip==0:       
            Y=np.append(Y,u1[rp])           
            
        vpx[:-1]=(u1[1:]-u1[:-1])
        mpx = b_x_half*mpx+a_x_half*vpx
        pxx= vpx+mpx

        vpxx[1:]= (pxx[1:]-pxx[:-1])
        mpxx = b_x*mpxx+a_x*vpxx 
        dpxx= vpxx+mpxx

        u2=2*u1-u0+kappa*dpxx-Sp*So[it]
        u2[0]=0
        u2[-1]=0    

        u0[:]=u1[:]
        u1[:]=u2[:]  
        
        #energy[it]=np.sum(0.5*u1*u1*v*v/1000000)
        
    return So,Y,NT,nx,dt


def kerF(n,var):
    x = np.linspace(-5,5,n)
    x = np.exp(-0.5*((x/var)**2))
    x = x/(var*((2*np.pi)**0.5))
    return x

def smoothV(v,kernel_dimension,kerne_variace):
    kernel=kerF(kernel_dimension,kerne_variace)
    kernel=kernel/np.sum(kernel)
    k_smooth=np.empty(v.size)
    k1=v[0]*np.ones(int((kernel_dimension-1)/2))
    k2=v[-1]*np.ones(int((kernel_dimension-1)/2))
    k_extend=np.append(k1,v)
    k_extend=np.append(k_extend,k2)
    for i in range(v.size):
        k_smooth[i]=np.dot(kernel,k_extend[i:i+kernel_dimension])
    return k_smooth


# model
def customRelu1(x):
    x=activations.relu(x,alpha=0.0, max_value=1.0, threshold=0.0)
    return x



def MLaplace(u1,mpx,mpxx):
        vpx = DRlayers(u1)
        mpx1 = Mlayers([input_bh,mpx])
        mpx2 = Mlayers([input_ah,vpx])
        mpx = Alayers([mpx1,mpx2])
        pxx = Alayers([vpx,mpx])

        vpx = DLlayers(pxx)
        mpx1 = Mlayers([input_b,mpxx])
        mpx2 = Mlayers([input_a,vpx])
        mpxx = Alayers([mpx1,mpx2])
        pxx = Alayers([vpx,mpxx])
    
        return pxx,mpx,mpxx

def wave(u0,u1,pxx,source):
        x = Mlayers([pxx,velocitylocal])
        uu1= Alayers([u1,u1])
        x = Alayers([x,uu1])
        u2 = Slayers([x,u0])    
        u2 = Slayers([u2,source])    
        u2 = Mlayers([u2,input_BC])
        return u1,u2 

def read(trace,s,u1,timeskip,i):
        if i%timeskip==0 :
            value=Players(u1) 
            trace = Clayers([trace,value])     
            
        sourcei = Pickfunction(s)  
        s=CropS(s)     
        return trace,s,sourcei
    
def repeat(x):
    return backend.repeat_elements(x,rep=npml, axis=1)

def PML_pad(x):
    xl=PML_PickL(x)
    xr=PML_PickR(x)
    xl= PML_repeat(xl)
    xr= PML_repeat(xr) 
    return Clayers([xl,x,xr]) 


def create_network(nx,nt,dx,dt,Rp,timeskip):  
    
    global DLlayers,DRlayers,Alayers,Mlayers,Slayers,Clayers,Players,CropS,Pickfunction,PML_PickL,PML_PickR,PML_repeat
    global input_a,input_b,input_ah,input_bh, input_BC
    global velocitylocal
    
     
    constants =  np.zeros([1,nx,1],dtype=np.float32)
    fixed_input0 = Input(shape=(nx,1),tensor=tf.constant(constants))
    
    constants =  a_x.astype(np.float32).reshape(nx,1)
    input_a = Input(shape=(nx,1),tensor=tf.constant(constants))
    
    constants =  b_x.astype(np.float32).reshape(nx,1)
    input_b = Input(shape=(nx,1),tensor=tf.constant(constants))
    
    constants =  a_x_half.astype(np.float32).reshape(nx,1)
    input_ah =Input(shape=(nx,1),tensor=tf.constant(constants))
   
    constants =  b_x_half.astype(np.float32).reshape(nx,1)
    input_bh =Input(shape=(nx,1),tensor=tf.constant(constants))   
   
    constants=np.ones([nx])
    constants[0]=0 
    constants[-1]=0
    constants=constants.astype(np.float32).reshape(nx,1)
    input_BC = Input(shape=(nx,1),tensor=tf.constant(constants))    
    
    constants= np.ones([1,1])*dt/dx
    constants=constants.astype(np.float32)
    input_c = Input(shape=(1,1),tensor=tf.constant(constants)) 
    
    input_s = Input(shape=(nx,nt,1),name="Input_s") # source
    

    #layer
    wR=np.array([[[0]],[[-1]],[[1]]])
    wL=np.array([[[-1]],[[1]],[[0]]])
    
    DLlayers=layers.Conv1D(1,3,padding='same', activation=None,weights=[wL],use_bias=False,trainable=False,name='DL')
    DRlayers=layers.Conv1D(1,3,padding='same', activation=None,weights=[wR],use_bias=False,trainable=False,name='DR')
    
    
    I = layers.Lambda(lambda x: x,name='Identity',trainable=False)
    Alayers = layers.Add(name='Add',trainable=False)
    Mlayers = layers.Multiply(name='Multiply',trainable=False)
    Slayers = layers.Subtract(name='Subtract',trainable=False)
    Clayers = layers.Concatenate(axis=1,name='Concate',trainable=False)
    
    nnode=int(nx-2*npml)
    Players = layers.Lambda(lambda x: x[:,Rp:Rp+1],name="Pick_data",trainable=False)    #reciever 
    Local=layers.Dense(nnode,activation=customRelu1,use_bias=False,name='local_velocity')
    CropS=layers.Lambda(lambda x: x[:,:,1:,:],name="Crop_source",trainable=False)
    Pickfunction = layers.Lambda(lambda x: x[:,:,0:1,0],output_shape=[nx,1],name="Pick_source",trainable=False)
    
    PML_PickL = layers.Lambda(lambda x: x[:,0:1,0:],output_shape=[1,1],trainable=False) 
    PML_PickR = layers.Lambda(lambda x: x[:,-1:,0:],output_shape=[1,1],trainable=False) 
    PML_repeat = layers.Lambda(repeat,trainable=False)
    
    
    # initial 
    velocitylocal = Local(input_c)
    velocitylocal =  layers.Reshape([nnode,1],trainable=False)(velocitylocal)
    #velocitylocal = layers.UpSampling1D(size=2)(velocitylocal)
    velocitylocal = PML_pad(velocitylocal)
    velocitylocal = Mlayers([velocitylocal,velocitylocal])
    
    u0 = I(fixed_input0)
    mpx= I(fixed_input0)
    mpxx= I(fixed_input0)

    #first time step (consider u0,mpx,mpxx =0)
    trace = Players(u0)
    source = Pickfunction(input_s)
    stotal= CropS(input_s)
    u1 = Alayers([u0,source]) 
    u1 = Mlayers([u1,input_BC])
    
    for i in range(1,nt):
        [trace,stotal,source]=read(trace,stotal,u1,timeskip,i)
        [pxx,mpx,mpxx] = MLaplace(u1,mpx,mpxx)             
        [u0,u1] = wave(u0,u1,pxx,source)       

    output_tensor = I(trace)  # wave_field at recievers

    model = Model([input_c,input_a,input_b,input_ah,input_bh, input_s,input_BC,fixed_input0], output_tensor)
    return model

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from scipy import interpolate
data = np.loadtxt('mar-vel.dat')
data = np.reshape(data,[data.size])
data = np.transpose(np.reshape(data[:-4],[116,76]))
plt.imshow(data)
plt.show()

X = np.linspace(0,3000,76)
Y = np.linspace(0,9200,116)
f_vmar = interpolate.interp2d(Y,X,data,kind='linear')

Xnew =  np.linspace(0,3000,int(3000/25))
Ynew = np.linspace(0,9200,int(9200/25))
vmar=f_vmar(Ynew,Xnew)
print(vmar.shape)
plt.imshow(vmar)
plt.show()

Xnew0 =  np.linspace(0,3000,int(80))
Ynew0 = np.linspace(0,9200,int(9200/25))
vmar0=f_vmar(Ynew0,Xnew0)

v2=vmar0[:,int(6500/25)]
plt.plot(v2)

In [ ]:
v02.shape

In [ ]:
3000/25

In [ ]:
# parameter 
global npml,nPhyx

totaltime= 1   #[second] total time
dimension=1000    #[meter]     Dimension
PML_size=75           #[meter]     PML layer  
dx= 12.5             #[meter]     dx
f0= 14                       #[Hz]   peak source frequency 
rp= 80                      #[meter]   reciever position 
sp= 100                    #[meter]  source position 
timeskip= 5               # [n x dt] time sampling, 5 == 5*dt (n == n*dt) 


#generate data
#v0=model1.layers[numlayer['local_velocity']].get_weights()
#v0=np.array(v_network).reshape(int(nx_real))
#v1=np.pad(v0, int(PML_size/dx), mode='edge')
#[So1,Seismogram1,nt1,nx1,dt1]=data_gen(f0,totaltime,dimension,PML_size,dx,rp,sp,timeskip,v1)

v = velocity_profile(dimension,PML_size,dx) 

#v=np.pad(v2, int(PML_size/dx), mode='edge')
[So,Seismogram,nt,nx,dt]=data_gen(f0,totaltime,dimension,PML_size,dx,rp,sp,timeskip,v)

# create smooth initial velocity profile 
kernel_dimension=int((20*12.5)/dx +1) # dimension of smooth function
kerne_variace=2.8    # width of smooth function
v_smooth=smoothV(v,kernel_dimension,kerne_variace)

#create network
print('creating network ... ')
npml = int(round(PML_size/dx))
nx_real = int(nx-2*npml)

model1=create_network(nx,nt,dx,dt,int((rp+PML_size)/dx),timeskip)   #need pml parameter(global parameters) from data_gen function  
print('network is created.')

#create dictionary for model layers
numlayer={}
for i,l in enumerate(model1.layers):
    numlayer[l.name] = i
    #print(i,l.name, l.trainable) 

#set initial velocity to smooth
a=np.array(model1.layers[numlayer['local_velocity']].get_weights()).shape
model1.layers[numlayer['local_velocity']].set_weights(v_smooth[npml:-npml].reshape(a))

#change data shape in order to suit input, output shape of network
input_s0=np.zeros([1,nx,nt,1])
input_s0[0,int((sp+PML_size)/dx),:,0]=So[:]
yyyy=Seismogram.reshape([1,Seismogram.size,1])


In [ ]:
#numlayer={}
for i,l in enumerate(model1.layers):
    #numlayer[l.name] = i
    print(i,l.name, l.trainable) 

In [ ]:
v_predict=model1.predict(x=input_s0)
print(v_predict.shape)
v_predict0=np.array(v_predict).reshape([v_predict.shape[1]])
#v_predict0=np.repeat(v_predict0,2)
print(v_predict0.shape)

In [ ]:
# display

print('the number of node = ',nx)
print('time steps (irteration) = ',nt)
print('data sampling = ',Seismogram.size)
print('sampling time = ',timeskip*dt*1000, ' ms')
plt.plot(np.linspace(0,totaltime,Seismogram.size),Seismogram)
#plt.plot(np.linspace(0,totaltime,Seismogram.size),Seismogram1,label='Predicted')
#plt.plot(Seismogram[15:],label='True seismogram')
#plt.plot(Seismogram1[15:],label='Predicted')

plt.ylabel('amplitude', fontsize=15)
plt.xlabel(' time (s) ', fontsize=15)
plt.legend()
plt.savefig('seimogram1.png')
plt.show()
#x=np.arange(-PML_size,dimension+PML_size,dx)    
#plt.plot(x,v,label= 'true velocity')
#plt.plot(x,v_smooth,label= 'initial')
#plt.legend()
x=np.arange(0,dimension,dx)    
plt.plot(x,v[npml:-npml])
#plt.plot(x,v1[npml:-npml])
plt.ylabel('velocity (m/s)', fontsize=15)
plt.xlabel(' depth (m) ', fontsize=15)
plt.savefig('vmodel1.png')

plt.show()

plt.plot(np.linspace(0,totaltime,nt),So)
#plt.plot(x,v1[npml:-npml])
plt.ylabel(' amplitude ', fontsize=15)
plt.xlabel(' time (s) ', fontsize=15)

plt.show()

In [ ]:
# test with ture velocity 

a=np.array(model1.layers[numlayer['local_velocity']].get_weights()).shape
model1.layers[numlayer['local_velocity']].set_weights(v[npml:-npml].reshape(a))

model1.compile(optimizer=tf.optimizers.Adam(lr=40), loss='mse') 
model1.evaluate(x=input_s0,y=yyyy)   

In [ ]:
plt.plot(v[npml:-npml])

In [ ]:
#set initial new velocity profile
a=np.array(model1.layers[numlayer['local_velocity']].get_weights()).shape
model1.layers[numlayer['local_velocity']].set_weights(v_smooth[npml:-npml].reshape(a))

In [ ]:
learning_rate=[1,5,40,100]
velocity_misfit1 = np.zeros([4,11])
x0=np.linspace(0,5000,11)
for i in range(4):

    a=np.array(model1.layers[numlayer['local_velocity']].get_weights()).shape
    model1.layers[numlayer['local_velocity']].set_weights(v_smooth[npml:-npml].reshape(a))
    
    model1.compile(optimizer=tf.optimizers.Adam(lr=learning_rate[i]), loss='mse') 
    
    for j in range(10):
        v_network=model1.layers[numlayer['local_velocity']].get_weights()
        v_network=np.array(v_network).reshape([nx_real])
        v_network=np.minimum(v_network,np.ones(nx_real)*5000)
        velocity_misfit1[i,j]=sum(abs(v_network-v[npml:-npml]))/nx_real
        print(i,j,velocity_misfit1[i,j])
        history=model1.fit(x=input_s0,y=yyyy,epochs=500,verbose=0)   
        
    v_network=model1.layers[numlayer['local_velocity']].get_weights()
    v_network=np.array(v_network).reshape([nx_real])
    v_network=np.minimum(v_network,np.ones(nx_real)*5000)
    velocity_misfit1[i,10]=sum(abs(v_network-v[npml:-npml]))/nx_real    
    print(i,10,velocity_misfit1[i,10])
    
for i in range(4):
    label0='lr = '
    label0 = label0+str(learning_rate[i])
    plt.plot(x0,velocity_misfit1[i,:],'o-',label= label0)
plt.legend()    
plt.ylabel(' velocity error (m/s) ', fontsize=15)
plt.xlabel(' epoch ', fontsize=15)
plt.savefig('new003.png')
plt.show()



In [ ]:
learning_rate=[1,5,40,100]
loss = np.zeros([4,5000])
for i in range(0,4):
    
    a=np.array(model1.layers[numlayer['local_velocity']].get_weights()).shape
    model1.layers[numlayer['local_velocity']].set_weights(v_smooth[npml:-npml].reshape(a))
    
    model1.compile(optimizer=tf.optimizers.Adam(lr=learning_rate[i]), loss='mse') 
    print(i)
    history=model1.fit(x=input_s0,y=yyyy,epochs=5000,verbose=0)   
    loss[i,:]=history.history['loss']
    




In [ ]:
np.log10(10)

In [ ]:
x0=np.arange(1,5000,250)
for i in range(4):
    label0='lr = '
    label0 = label0+str(learning_rate[i])
    plt.plot(x0,np.log10(loss[i,:])[::250],'-o',label= label0)
    plt.legend()    
    plt.ylabel(' $log_{10}$(loss) ', fontsize=15)
    plt.xlabel(' epoch ', fontsize=15)
plt.savefig('loss001.png')

In [ ]:
loss[i,:].shape

In [ ]:
a=np.arange(1,10)
print(a[::3])

In [ ]:
for i in range(4):
    label0='lr = '
    label0 = label0+str(learning_rate[i])
    plt.plot(x0,velocity_misfit1[i,:],'o-',label= label0)
plt.legend()    
plt.ylabel(' velocity error (m/s) ', fontsize=15)
plt.xlabel(' epoch ', fontsize=15)
plt.savefig('new01.png')


In [ ]:
for i in range(4):
    label0='lr = '
    label0 = label0+str(learning_rate[i])
    plt.plot(velocity_misfit[i,:],label= label0)
plt.legend()    

In [ ]:
model1.compile(optimizer=optimizers.Adam(lr=100), loss='mse') 

In [ ]:
#train  
history=model1.fit(x=input_s0,y=yyyy,epochs=500,verbose=1)   

In [ ]:
v_network=model1.layers[numlayer['local_velocity']].get_weights()
v_network=np.array(v_network).reshape(int(nx_real))
v_network=np.minimum(v_network,np.ones(nx_real)*5000)
#plot
x=np.linspace(0,1000,int(nx_real))
plt.plot(x,v_network,label= 'predicted velocity')
plt.plot(x,v[npml:-npml],label= 'true velocity')  
plt.plot(x,v_smooth[npml:-npml], '--', label= 'initial velocity')  
plt.legend()
plt.ylabel(' velocity (m/s)', fontsize=15)
plt.xlabel(' depth (m) ', fontsize=15)
#plt.savefig('m1lr100ep5000.png')
sum(abs(v_network-v[npml:-npml]))/nx_real


In [ ]:
plt.plot(history.history['loss'])

In [ ]:
# in case that we want to try other initial velocity  !!!!!!

kernel_dimension=20*8+1  # dimension of smooth function
kerne_variace=10    # width of smooth function
v_smooth2=smoothV(v,kernel_dimension,kerne_variace)

#v_smooth=np.ones(v_smooth.shape)*np.mean(v)

#plot
x=np.arange(-PML_size,dimension+PML_size,dx)    
plt.plot(x,v)
plt.plot(x,v_smooth2)
plt.show()

#set initial new velocity profile
a=np.array(model1.layers[numlayer['local_velocity']].get_weights()).shape
model1.layers[numlayer['local_velocity']].set_weights(v_smooth2[npml:-npml].reshape(a))

# test

In [ ]:
input_1 = Input(shape=(4,)) 


x = layers.Dense(4)(input_1)  

for i in range(3):
    x = layers.Dense(4)(x)  

model1 = Model([input_1], x)

model1.summary()

In [ ]:
input_1 = Input(shape=(4,)) 

A=layers.Dense(4)

x = A(input_1)  

for i in range(3):
    x = A(x)  

model2 = Model([input_1], x)
model2.summary()

In [ ]:
#Rp=2

#constants =  np.zeros([5,1],dtype=np.float32)
#fixed_input0 = Input(shape=(5,1),tensor=tf.constant(constants))

I = layers.Lambda(lambda x: x,name='Identity',trainable=False)
#Players = layers.Lambda(lambda x: x[:,Rp:Rp+1],name="Pick_data",trainable=False)    #reciever 
#Pickfunction = layers.Lambda(lambda x: x[:,:,0:1,0],output_shape=[nx,1],name="Pick_source",trainable=False)

input_1 = Input(shape=(5,1)) 

output_tensor = I(input_1)  

model123 = Model([input_1], output_tensor)


In [ ]:
x=np.random.randint(1,10,[1,5,1])

y=model123.predict(x)

print(x ,'\n\n')
print(y)

In [ ]:
#Mlayers([input_ah,vpx])

Mlayers = layers.Multiply(name='Multiply',trainable=False)

constants = np.array([1,2,3,4,5],dtype=np.float32).reshape(5,1)

fixed_input0 = Input(shape=(5,1),tensor=tf.constant(constants))
input_1 = Input(shape=(5,1)) 

output_tensor = Mlayers([fixed_input0,input_1])  

model = Model([input_1,fixed_input0], output_tensor)

In [ ]:
x=np.random.randint(1,5,[2,5,1])

y=model.predict(x)

print(x ,'\n\n')
print(y)